In [12]:
%load_ext autoreload
%autoreload 2
import TCDF
import argparse
import torch
import pandas as pd
import numpy as np
import networkx as nx
import pylab
import copy
import matplotlib.pyplot as plt
import os
import sys
import time
from tqdm.auto import tqdm

# Set CUDA gpu device
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.chdir(os.path.dirname(sys.argv[0])) #uncomment this line to run in VSCode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def getextendeddelays(gtfile, columns):
    """Collects the total delay of indirect causal relationships."""
#     print(columns)
    gtdata = pd.read_csv(gtfile, header=None)

    readgt=dict()
    effects = gtdata[1]
    causes = gtdata[0]
    delays = gtdata[2]
    gtnrrelations = 0
    pairdelays = dict()
    for k in range(len(columns)):
        readgt[k]=[]
    for i in range(len(effects)):
        key=effects[i]
        value=causes[i]
        readgt[key].append(value)
        pairdelays[(key, value)]=delays[i]
        gtnrrelations+=1
    
    g = nx.DiGraph()
    g.add_nodes_from(readgt.keys())
    for e in readgt:
        cs = readgt[e]
        for c in cs:
            g.add_edge(c, e)

    extendedreadgt = copy.deepcopy(readgt)
    
    for c1 in range(len(columns)):
        for c2 in range(len(columns)):
            paths = list(nx.all_simple_paths(g, c1, c2, cutoff=2)) #indirect path max length 3, no cycles
            
            if len(paths)>0:
                for path in paths:
                    for p in path[:-1]:
                        if p not in extendedreadgt[path[-1]]:
                            extendedreadgt[path[-1]].append(p)
                            
    extendedgtdelays = dict()
    for effect in extendedreadgt:
        causes = extendedreadgt[effect]
        for cause in causes:
            if (effect, cause) in pairdelays:
                delay = pairdelays[(effect, cause)]
                extendedgtdelays[(effect, cause)]=[delay]
            else:
                #find extended delay
                paths = list(nx.all_simple_paths(g, cause, effect, cutoff=2)) #indirect path max length 3, no cycles
                extendedgtdelays[(effect, cause)]=[]
                for p in paths:
                    delay=0
                    for i in range(len(p)-1):
                        delay+=pairdelays[(p[i+1], p[i])]
                    extendedgtdelays[(effect, cause)].append(delay)

    return extendedgtdelays, readgt, extendedreadgt

def evaluate(gtfile, validatedcauses, columns, verbose=True):
    """Evaluates the results of TCDF by comparing it to the ground truth graph, and calculating precision, recall and F1-score. F1'-score, precision' and recall' include indirect causal relationships."""
    extendedgtdelays, readgt, extendedreadgt = getextendeddelays(gtfile, columns)
    FP=0
    FPdirect=0
    TPdirect=0
    TP=0
    FN=0
    FPs = []
    FPsdirect = []
    TPsdirect = []
    TPs = []
    FNs = []
    for key in readgt:
        for v in validatedcauses[key]:
            if v not in extendedreadgt[key]:
                FP+=1
                FPs.append((key,v))
            else:
                TP+=1
                TPs.append((key,v))
            if v not in readgt[key]:
                FPdirect+=1
                FPsdirect.append((key,v))
            else:
                TPdirect+=1
                TPsdirect.append((key,v))
        for v in readgt[key]:
            if v not in validatedcauses[key]:
                FN+=1
                FNs.append((key, v))
        
    
    precision = recall = 0.

    if float(TP+FP)>0:
        precision = TP / float(TP+FP)
    
    if float(TP + FN)>0:
        recall = TP / float(TP + FN)
    
    if (precision + recall) > 0:
        F1 = 2 * (precision * recall) / (precision + recall)
    else:
        F1 = 0.

    precision = recall = 0.
    if float(TPdirect+FPdirect)>0:
        precision = TPdirect / float(TPdirect+FPdirect)
    
    if float(TPdirect + FN)>0:
        recall = TPdirect / float(TPdirect + FN)
    
    if (precision + recall) > 0:
        F1direct = 2 * (precision * recall) / (precision + recall)
    else:
        F1direct = 0.
    if verbose:
        print("Total False Positives': ", FP)
        print("Total True Positives': ", TP)
        print("Total False Negatives: ", FN)
        print("Total Direct False Positives: ", FPdirect)
        print("Total Direct True Positives: ", TPdirect)
        print("TPs': ", TPs)
        print("FPs': ", FPs)
        print("TPs direct: ", TPsdirect)
        print("FPs direct: ", FPsdirect)
        print("FNs: ", FNs)
        print("Precision': ", precision)
        print("Recall': ", recall)
        print("F1' score: ", F1,"(includes direct and indirect causal relationships)")
        print("Precision: ", precision)
        print("Recall: ", recall)
        print("F1 score: ", F1direct,"(includes only direct causal relationships)")
    return FP, TP, FPdirect, TPdirect, FN, FPs, FPsdirect, TPs, TPsdirect, FNs, F1, F1direct

def evaluatedelay(extendedgtdelays, alldelays, TPs, receptivefield):
    """Evaluates the delay discovery of TCDF by comparing the discovered time delays with the ground truth."""
    zeros = 0
    total = 0.
    for i in range(len(TPs)):
        tp=TPs[i]
        discovereddelay = alldelays[tp]
        gtdelays = extendedgtdelays[tp]
        for d in gtdelays:
            if d <= receptivefield:
                total+=1.
                error = d - discovereddelay
                if error == 0:
                    zeros+=1
                
            else:
                next
           
    if zeros==0:
        return 0.
    else:
        return zeros/float(total)
    
    
def runTCDF(datafile, args):
    """Loops through all variables in a dataset and return the discovered causes, time delays, losses, attention scores and variable names."""
    df_data = pd.read_csv(datafile, header=0)
    allcauses = dict()
    alldelays = dict()
    allreallosses=dict()
    allscores=dict()

    columns = list(df_data)
    for c in columns:
        idx = df_data.columns.get_loc(c)
        causes, causeswithdelay, realloss, scores = TCDF.findcauses(c, cuda=args.cuda, epochs=args.epochs, 
            kernel_size=args.kernel_size, layers=args.hidden_layers+1, log_interval=args.log_interval, 
            lr=args.learning_rate, optimizername=args.optimizer,
            seed=args.seed, dilation_c=args.dilation_coefficient, 
            significance=args.significance, file=datafile, verbose=args.verbose)

        allscores[idx]=scores
        allcauses[idx]=causes
        alldelays.update(causeswithdelay)
        allreallosses[idx]=realloss

    return allcauses, alldelays, allreallosses, allscores, columns

def plotgraph(stringdatafile,alldelays,columns):
    """Plots a temporal causal graph showing all discovered causal relationships annotated with the time delay between cause and effect."""
    G = nx.DiGraph()
    for c in columns:
        G.add_node(c)
    for pair in alldelays:
        p1,p2 = pair
        nodepair = (columns[p2], columns[p1])

        G.add_edges_from([nodepair],weight=alldelays[pair])
    
    edge_labels=dict([((u,v,),d['weight'])
                    for u,v,d in G.edges(data=True)])
    
    pos=nx.circular_layout(G)
    nx.draw_networkx_edge_labels(G,pos,edge_labels=edge_labels)
    nx.draw(G,pos, node_color = 'white', edge_color='black',node_size=1000,with_labels = True)
    ax = plt.gca()
    ax.collections[0].set_edgecolor("#000000") 

    plt.show()

def main(datafiles, args, evaluation):
    if evaluation:
        totalF1direct = [] #contains F1-scores of all datasets
        totalF1 = [] #contains F1'-scores of all datasets

        receptivefield=1
        for l in range(0, levels):
            receptivefield+=(kernel_size-1) * dilation_c**(l)

    exp_records = []
    for datafile in datafiles.keys(): 
        stringdatafile = str(datafile)
        if '/' in stringdatafile:
            stringdatafile = str(datafile).rsplit('/', 1)[1]
        if args.verbose:
            print("\n Dataset: ", stringdatafile)

        # run TCDF
        allcauses, alldelays, allreallosses, allscores, columns = runTCDF(datafile, args) #results of TCDF containing indices of causes and effects
        d = {
            "allcauses": allcauses,
            "alldelays": alldelays,
            "allreallosses": allreallosses,
            "allscores": allscores,
            "columns": columns
        }
        if args.verbose:
            print("\n===================Results for", stringdatafile,"==================================")
            for pair in alldelays:
                print(columns[pair[1]], "causes", columns[pair[0]],"with a delay of",alldelays[pair],"time steps.")

        

        if evaluation:
            # evaluate TCDF by comparing discovered causes with ground truth
            if args.verbose:
                print("\n===================Evaluation for", stringdatafile,"===============================")
            FP, TP, FPdirect, TPdirect, FN, FPs, FPsdirect, TPs, TPsdirect, FNs, F1, F1direct \
                = evaluate(datafiles[datafile], allcauses, columns, verbose=args.verbose)
            # Update to records
            for k, v in zip(
                ["FP", "TP", "FPdirect", "TPdirect", "FN", "FPs", "FPsdirect", 
                 "TPs", "TPsdirect", "FNs", "F1", "F1direct"],
                [FP, TP, FPdirect, TPdirect, FN, FPs, FPsdirect, TPs, TPsdirect, FNs, F1, F1direct]
            ):
                d[k] = v
            totalF1.append(F1)
            totalF1direct.append(F1direct)

            # evaluate delay discovery
            extendeddelays, readgt, extendedreadgt = getextendeddelays(datafiles[datafile], columns)
            percentagecorrect = evaluatedelay(extendeddelays, alldelays, TPs, receptivefield)*100
            if args.verbose:
                print("Percentage of delays that are correctly discovered: ", percentagecorrect,"%")
        if args.verbose:
            print("==================================================================================")
        # Save to records
        exp_records.append(d)
        if args.plot:
            plotgraph(stringdatafile, alldelays, columns)

    # In case of multiple datasets, calculate average F1-score over all datasets and standard deviation
    if len(datafiles.keys())>1 and evaluation and args.verbose:  
        print("\nOverall Evaluation: \n")      
        print("F1' scores: ")
        for f in totalF1:
            print(f)
        print("Average F1': ", np.mean(totalF1))
        print("Standard Deviation F1': ", np.std(totalF1),"\n")
        print("F1 scores: ")
        for f in totalF1direct:
            print(f)
        print("Average F1: ", np.mean(totalF1direct))
        print("Standard Deviation F1: ", np.std(totalF1direct))
    return exp_records

In [4]:
tcdf_data_path = 'data/Finance/'
timeseries_files = ['manyinputs_returns30007000_header.csv',
                    'random-rels_20_1A_returns30007000_header.csv',
                    'random-rels_20_1B_returns30007000_header.csv',
                    'random-rels_20_1C_returns30007000_header.csv',
                    'random-rels_20_1D_returns30007000_header.csv',
                    'random-rels_20_1E_returns30007000_header.csv',
                    'random-rels_40_1_returns30007000_header.csv',
                    'random-rels_40_1_3_returns30007000_header.csv',
                    'random-rels_20_1_3_returns30007000_header.csv'
                   ]
gt_files = ['manyinputs.csv',
            'random-rels_20_1A.csv',
            'random-rels_20_1B.csv',
            'random-rels_20_1C.csv',
            'random-rels_20_1D.csv',
            'random-rels_20_1E.csv',
            'random-rels_20_1_3.csv',
            'random-rels_40_1.csv',
            'random-rels_40_1_3.csv'
           ]
s = ""
for i in range(len(timeseries_files)):
    s += tcdf_data_path+timeseries_files[i]+"="+tcdf_data_path+gt_files[i]
    if i != len(timeseries_files)-1:
        s += ","
print(s)

data/Finance/manyinputs_returns30007000_header.csv=data/Finance/manyinputs.csv,data/Finance/random-rels_20_1A_returns30007000_header.csv=data/Finance/random-rels_20_1A.csv,data/Finance/random-rels_20_1B_returns30007000_header.csv=data/Finance/random-rels_20_1B.csv,data/Finance/random-rels_20_1C_returns30007000_header.csv=data/Finance/random-rels_20_1C.csv,data/Finance/random-rels_20_1D_returns30007000_header.csv=data/Finance/random-rels_20_1D.csv,data/Finance/random-rels_20_1E_returns30007000_header.csv=data/Finance/random-rels_20_1E.csv,data/Finance/random-rels_40_1_returns30007000_header.csv=data/Finance/random-rels_20_1_3.csv,data/Finance/random-rels_40_1_3_returns30007000_header.csv=data/Finance/random-rels_40_1.csv,data/Finance/random-rels_20_1_3_returns30007000_header.csv=data/Finance/random-rels_40_1_3.csv


In [10]:
tcdf_data_path = 'data/fMRI/'
s = ""
for i in range(1, 29, 1):
    data = pd.read_csv(tcdf_data_path + f"timeseries{i}.csv", header=0)
    if data.shape[0] > 1000:
        s += tcdf_data_path + f"timeseries{i}.csv" + "=" + tcdf_data_path + f"sim{i}_gt_processed.csv"
        s += ","
s = s[:-1]
print(s)

data/fMRI/timeseries5.csv=data/fMRI/sim5_gt_processed.csv,data/fMRI/timeseries6.csv=data/fMRI/sim6_gt_processed.csv,data/fMRI/timeseries7.csv=data/fMRI/sim7_gt_processed.csv,data/fMRI/timeseries9.csv=data/fMRI/sim9_gt_processed.csv,data/fMRI/timeseries19.csv=data/fMRI/sim19_gt_processed.csv,data/fMRI/timeseries20.csv=data/fMRI/sim20_gt_processed.csv


In [3]:
s='data/Finance/manyinputs_returns30007000_header.csv=data/Finance/manyinputs.csv'

In [11]:
from args import build_argparser
parser = build_argparser()
args = parser.parse_args(['--ground_truth', s, 
#                           '--plot', 
                          '--hidden_layers', '3', 
                          '--log_interval', '100',
                          '--learning_rate', '0.01',
                          '--epochs', '500',
#                           '--verbose',
                          '--cuda'])
print("Arguments:", args)

Arguments: Namespace(cuda=True, data=None, dilation_coefficient=4, epochs=500, ground_truth={'data/fMRI/timeseries5.csv': 'data/fMRI/sim5_gt_processed.csv', 'data/fMRI/timeseries6.csv': 'data/fMRI/sim6_gt_processed.csv', 'data/fMRI/timeseries7.csv': 'data/fMRI/sim7_gt_processed.csv', 'data/fMRI/timeseries9.csv': 'data/fMRI/sim9_gt_processed.csv', 'data/fMRI/timeseries19.csv': 'data/fMRI/sim19_gt_processed.csv', 'data/fMRI/timeseries20.csv': 'data/fMRI/sim20_gt_processed.csv'}, hidden_layers=3, kernel_size=4, learning_rate=0.01, log_interval=100, optimizer='Adam', plot=False, seed=1111, significance=0.8, train_test_split=0.8, verbose=False)


In [16]:
for sign in np.arange(0.1, 1.0, 0.1):
    if sign==0.1:
        continue
    args.significance = sign
    print("Arguments:", args)

    if torch.cuda.is_available():
        if not args.cuda:
            print("WARNING: You have a CUDA device, you should probably run with --cuda to speed up training.")
    if args.kernel_size != args.dilation_coefficient:
        print("WARNING: The dilation coefficient is not equal to the kernel size. Multiple paths can lead to the same delays. Set kernel_size equal to dilation_c to have exaxtly one path for each delay.")

    kernel_size = args.kernel_size
    levels = args.hidden_layers+1
    nrepochs = args.epochs
    learningrate = args.learning_rate
    optimizername = args.optimizer
    dilation_c = args.dilation_coefficient
    loginterval = args.log_interval
    seed=args.seed
    cuda=args.cuda
    significance=args.significance

    tic = time.time()
    if args.ground_truth is not None:
        datafiles = args.ground_truth
        exp_records = main(datafiles, args, True)

    else:
        datafiles = dict()
        for dataset in args.data:
            datafiles[dataset]=""
        exp_records = main(datafiles, args, False)
    toc = time.time() - tic

    import datetime
    import os
    import pickle
    # Use the timezone in my location.
    local_tz = datetime.timezone(datetime.timedelta(hours=8))
    time_str = datetime.datetime.now(local_tz).strftime("%Y%m%d_%H%M%S")
    fname = os.path.join("temp_results", "tcdf", "tcdf_fmri", f"exp_records_sign{args.significance:.1f}_{time_str}.pkl")
    print(fname)
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    with open(fname, "wb") as f:
    #     pickle.dump(exp_rets, f)
        pickle.dump({'exp_records': exp_records, 'time': toc}, f)

Arguments: Namespace(cuda=True, data=None, dilation_coefficient=4, epochs=500, ground_truth={'data/fMRI/timeseries5.csv': 'data/fMRI/sim5_gt_processed.csv', 'data/fMRI/timeseries6.csv': 'data/fMRI/sim6_gt_processed.csv', 'data/fMRI/timeseries7.csv': 'data/fMRI/sim7_gt_processed.csv', 'data/fMRI/timeseries9.csv': 'data/fMRI/sim9_gt_processed.csv', 'data/fMRI/timeseries19.csv': 'data/fMRI/sim19_gt_processed.csv', 'data/fMRI/timeseries20.csv': 'data/fMRI/sim20_gt_processed.csv'}, hidden_layers=3, kernel_size=4, learning_rate=0.01, log_interval=100, optimizer='Adam', plot=False, seed=1111, significance=0.2, train_test_split=0.8, verbose=False)
/workspace/dycause-efficiency-analysis/temp_results/tcdf/tcdf_fmri/exp_records_sign0.2_20211011_114215.pkl
Arguments: Namespace(cuda=True, data=None, dilation_coefficient=4, epochs=500, ground_truth={'data/fMRI/timeseries5.csv': 'data/fMRI/sim5_gt_processed.csv', 'data/fMRI/timeseries6.csv': 'data/fMRI/sim6_gt_processed.csv', 'data/fMRI/timeseries7.c